In [105]:
import pandas as pd
import numpy as np
import tensorflow as tf
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
pd.set_option('display.max_columns', None)
DATA_PATH = './data/'

In [106]:
league_data = pd.read_csv('data/train_data_part1.csv')
# Scale DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
league_data.MW = league_data.MW.astype(float)

for col in cols:
    league_data[col] = league_data[col] / league_data.MW
league_data.tail()

,Unnamed: 0,Date,season,HomeTeam,AwayTeam,FTR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,B365H,B365D,B365A,HM1,HM2,HM3,HM4,HM5,AM1,AM2,AM3,AM4,AM5,MW,gameId,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
4935,4935,2020-07-26,1920,Leicester,Man United,A,67.0,62.0,41.0,36.0,1.631579,1.657895,3.25,3.50,2.20,L,W,L,D,W,D,W,D,W,W,38.0,4936,LWLDW,DWDWW,7,11,0.684211,0.684211,-0.026316,-0.105263,3
4936,4936,2020-07-26,1920,Man City,Norwich,H,97.0,25.0,35.0,71.0,2.052632,0.552632,1.08,11.00,26.00,W,W,W,W,L,L,L,L,L,L,38.0,4937,WWWWL,LLLLL,12,0,1.631579,-1.210526,1.500000,0.315789,-17
4937,4937,2020-07-26,1920,Newcastle,Liverpool,A,38.0,81.0,57.0,32.0,1.157895,2.526316,7.50,5.25,1.36,D,L,L,L,D,W,L,D,W,W,38.0,4938,DLLLD,WLDWW,2,10,-0.500000,1.289474,-1.368421,-0.210526,11
4938,4938,2020-07-26,1920,Southampton,Sheffield United,H,51.0,38.0,57.0,38.0,1.289474,1.421053,2.15,3.50,3.30,W,D,D,D,W,L,L,W,W,D,38.0,4939,WDDDW,LLWWD,9,7,-0.157895,0.000000,-0.131579,0.052632,-2
4939,4939,2020-07-26,1920,West Ham,Aston Villa,D,49.0,40.0,57.0,64.0,1.000000,0.894737,3.00,3.60,2.30,D,W,W,L,D,W,D,W,L,L,38.0,4940,DWWLD,WDWLL,8,7,-0.210526,-0.631579,0.105263,0.026316,-8


In [107]:
league_data.drop(['Unnamed: 0', 'Date', 'HomeTeam', 'AwayTeam', 
                 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HM1', 'HM2', 'HM3', 
                 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5', 'HTFormPts',
                 'ATFormPts', 'MW', 'HTFormPtsStr', 'ATFormPtsStr'], 1, inplace=True)
league_data.columns

Index(['season', 'FTR', 'HTP', 'ATP', 'B365H', 'B365D', 'B365A', 'gameId',
       'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP'],
      dtype='object')

In [108]:
EMA_data = pd.read_csv('data/train_data_part2.csv')
EMA_data.tail()

,Unnamed: 0,f_DateHome,f_seasonHome,gameId,HomeTeam,homeGame_x,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,f_goalsForHome,f_halfTimeGoalsAgainstHome,f_halfTimeGoalsForHome,f_redsAgainstHome,f_redsForHome,f_shotsAgainstHome,f_shotsForHome,f_shotsOnTargetAgainstHome,f_shotsOnTargetForHome,f_yellowsAgainstHome,f_yellowsForHome,f_DateAway,f_seasonAway,AwayTeam,homeGame_y,f_cornersAgainstAway,f_cornersForAway,f_freesAgainstAway,f_freesForAway,f_goalsAgainstAway,f_goalsForAway,f_halfTimeGoalsAgainstAway,f_halfTimeGoalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway,f_yellowsAgainstAway,f_yellowsForAway
4627,4684,2019-11-23,1920,4686,Everton,1,4.565659,6.002476,10.557303,11.607718,1.295098,1.288115,0.446414,0.649212,0.098173,0.108450,9.864738,13.110057,3.619589,4.498122,1.849922,1.457246,2019-11-23,1920,Norwich,0,6.225871,4.859451,10.158318,10.465967,1.943386,0.889192,0.889350,0.374763,0.069598,0.033448,14.703841,11.103013,5.259448,3.439076,1.311019,1.576155
4628,4685,2019-11-23,1920,4687,Watford,1,5.551670,4.676105,8.990351,11.080792,1.731395,1.044866,0.835796,0.388996,0.040900,0.111778,13.255541,11.782914,5.116522,3.665730,1.514075,2.139763,2019-11-23,1920,Burnley,0,6.125178,4.624889,10.010339,9.744711,1.587280,1.277872,0.761675,0.591053,0.083338,0.014158,15.168061,10.171129,4.825411,3.546055,1.173971,1.876547
4629,4686,2019-11-23,1920,4688,Man City,1,2.394241,8.241461,8.321039,9.099733,0.796715,2.561447,0.430477,1.210825,0.030523,0.051923,6.672742,19.254371,2.803048,6.796080,1.463633,1.536028,2019-11-23,1920,Chelsea,0,3.412852,5.869358,11.010736,8.960500,1.160939,1.846676,0.452702,0.859091,0.006360,0.008065,9.071821,16.309098,3.200263,5.812427,1.681836,1.526415
4630,4687,2019-11-24,1920,4689,Sheffield United,1,6.506138,6.033564,8.084640,10.255848,0.718892,1.103651,0.316085,0.468612,0.000000,0.077736,11.463558,10.422463,3.458262,3.160077,1.402459,1.858845,2019-11-24,1920,Man United,0,4.366679,5.596742,11.869497,10.826574,1.169153,1.471642,0.415808,0.834202,0.044394,0.047382,11.412037,14.247559,4.033247,5.391701,2.129670,1.980482
4631,4688,2019-11-24,1920,4690,Aston Villa,1,6.980479,4.315249,11.577326,10.922120,1.902351,1.001041,0.641784,0.439997,0.140163,0.103278,15.400326,11.139602,4.965691,3.893277,1.708601,1.789916,2019-11-24,1920,Newcastle,0,6.074955,4.169981,9.144160,10.013072,1.334469,1.146110,0.511650,0.623525,0.016440,0.091794,13.178900,11.479283,4.157185,3.785123,1.609159,1.545347


In [109]:
EMA_data.drop(['Unnamed: 0', 'f_DateHome', 'f_seasonHome', 'HomeTeam',
               'homeGame_x', 'f_yellowsAgainstAway', 'f_yellowsForAway',
               'f_yellowsAgainstHome', 'f_yellowsForHome', 'f_DateAway', 'f_seasonAway', 
               'AwayTeam', 'homeGame_y'], 1, inplace=True)
EMA_data.columns

Index(['gameId', 'f_cornersAgainstHome', 'f_cornersForHome',
       'f_freesAgainstHome', 'f_freesForHome', 'f_goalsAgainstHome',
       'f_goalsForHome', 'f_halfTimeGoalsAgainstHome',
       'f_halfTimeGoalsForHome', 'f_redsAgainstHome', 'f_redsForHome',
       'f_shotsAgainstHome', 'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_halfTimeGoalsAgainstAway',
       'f_halfTimeGoalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway'],
      dtype='object')

In [110]:
TRAINING = True
df = pd.merge(EMA_data, league_data, left_on='gameId', right_index=True)
df.head()
df.drop(['gameId_x','gameId_y'], 1, inplace=True)

df.tail()

,gameId,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,f_goalsForHome,f_halfTimeGoalsAgainstHome,f_halfTimeGoalsForHome,f_redsAgainstHome,f_redsForHome,f_shotsAgainstHome,f_shotsForHome,f_shotsOnTargetAgainstHome,f_shotsOnTargetForHome,f_cornersAgainstAway,f_cornersForAway,f_freesAgainstAway,f_freesForAway,f_goalsAgainstAway,f_goalsForAway,f_halfTimeGoalsAgainstAway,f_halfTimeGoalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway,season,FTR,HTP,ATP,B365H,B365D,B365A,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
4627,4686,4.565659,6.002476,10.557303,11.607718,1.295098,1.288115,0.446414,0.649212,0.098173,0.108450,9.864738,13.110057,3.619589,4.498122,6.225871,4.859451,10.158318,10.465967,1.943386,0.889192,0.889350,0.374763,0.069598,0.033448,14.703841,11.103013,5.259448,3.439076,1920,A,0.615385,1.153846,2.30,3.4,3.10,-1.153846,-0.076923,-0.538462,0.000000,-4
4628,4687,5.551670,4.676105,8.990351,11.080792,1.731395,1.044866,0.835796,0.388996,0.040900,0.111778,13.255541,11.782914,5.116522,3.665730,6.125178,4.624889,10.010339,9.744711,1.587280,1.277872,0.761675,0.591053,0.083338,0.014158,15.168061,10.171129,4.825411,3.546055,1920,H,1.923077,2.000000,1.44,5.0,6.50,1.384615,1.000000,-0.076923,-0.461538,-2
4629,4688,2.394241,8.241461,8.321039,9.099733,0.796715,2.561447,0.430477,1.210825,0.030523,0.051923,6.672742,19.254371,2.803048,6.796080,3.412852,5.869358,11.010736,8.960500,1.160939,1.846676,0.452702,0.859091,0.006360,0.008065,9.071821,16.309098,3.200263,5.812427,1920,D,1.307692,1.230769,3.50,3.4,2.10,0.307692,0.000000,0.076923,0.153846,12
4630,4689,6.506138,6.033564,8.084640,10.255848,0.718892,1.103651,0.316085,0.468612,0.000000,0.077736,11.463558,10.422463,3.458262,3.160077,4.366679,5.596742,11.869497,10.826574,1.169153,1.471642,0.415808,0.834202,0.044394,0.047382,11.412037,14.247559,4.033247,5.391701,1920,H,0.846154,1.153846,2.05,3.4,3.70,0.076923,-0.615385,-0.307692,-0.307692,5
4631,4690,6.980479,4.315249,11.577326,10.922120,1.902351,1.001041,0.641784,0.439997,0.140163,0.103278,15.400326,11.139602,4.965691,3.893277,6.074955,4.169981,9.144160,10.013072,1.334469,1.146110,0.511650,0.623525,0.016440,0.091794,13.178900,11.479283,4.157185,3.785123,1920,D,1.071429,2.000000,13.00,8.0,1.16,-0.571429,1.285714,-0.928571,-0.357143,12


In [111]:
TRAINING = True


In [112]:
if TRAINING:
    training_data = df.loc[df['season'] != 1920].reset_index(drop=True)
    testing_data = df.loc[df['season'] == 1920].reset_index(drop=True)
    X = training_data.drop(['FTR','season','gameId'], 1)
    Y = training_data['FTR']

    X_test = testing_data.drop(['FTR','season','gameId'], 1)
    y_test = testing_data['FTR']
    
if not TRAINING:
    X_test = df.drop(['FTR','season','gameId'], 1)
    y_test = df['FTR']

In [113]:
def transform_results(results):
    transformed = []
    for i in range(len(results)):
        if results[i] == 'H':
            transformed.append(0)
        elif results[i] == 'A':
            transformed.append(2)
        else:
            transformed.append(1)
    return np.array(transformed)

if TRAINING:
    Y = transform_results(Y)
y_test = transform_results(y_test)

In [114]:
Y[0:100]

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 1, 0, 1, 2, 1, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 1, 2, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 1, 1, 0, 0, 2, 2, 2, 0, 1, 0, 0, 0, 0, 2, 2, 1, 0, 0, 0,
       0, 2, 0, 2, 1, 0, 2, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 2, 1, 0,
       1, 0, 0, 1, 2, 2, 1, 0, 2, 0, 1, 2])

In [ ]:
if TRAINING:
    args = {'objective': 'multi:softmax',
            'learning_rate': 0.01,
            'tree_method': 'gpu_hist',
            'max_depth':20,
            'num_classes':3,
            'n_estimators': 400}
    clf = xgb.XGBClassifier(**args)

    clf.fit(X,
        Y,
        eval_set=[(X, Y), (X_test, y_test)],
        verbose=True,

        early_stopping_rounds = 10)  


In [ ]:
y_preds = clf.predict_proba(X_test)
y_pred_argmax = [np.argmax(i) for i in y_preds]
labels = ['Home', 'Draw', 'Away']
print(classification_report(y_test, y_pred_argmax, target_names=labels))
print(confusion_matrix(y_test, y_pred_argmax))

In [123]:
y_preds = clf.predict_proba(X)
y_pred_argmax = [np.argmax(i) for i in y_preds]
labels = ['Home', 'Draw', 'Away']
print(classification_report(Y, y_pred_argmax, target_names=labels))
print(confusion_matrix(Y, y_pred_argmax))

              precision    recall  f1-score   support

        Home       0.59      0.86      0.70      2082
        Draw       0.36      0.07      0.11      1124
        Away       0.59      0.58      0.58      1297

    accuracy                           0.58      4503
   macro avg       0.51      0.50      0.47      4503
weighted avg       0.53      0.58      0.52      4503

[[1789   59  234]
 [ 764   75  285]
 [ 474   73  750]]


In [124]:
y_preds = clf.predict_proba(X_test)
y_preds

array([[0.35441118, 0.3160158 , 0.32957304],
       [0.3104017 , 0.31818262, 0.37141564],
       [0.31523523, 0.34776473, 0.33700004],
       [0.30904394, 0.34093451, 0.35002154],
       [0.34607604, 0.3321019 , 0.32182205],
       [0.35441118, 0.3160158 , 0.32957304],
       [0.31523523, 0.34776473, 0.33700004],
       [0.30305275, 0.3343251 , 0.3626221 ],
       [0.31523523, 0.34776473, 0.33700004],
       [0.4508531 , 0.29054853, 0.2585984 ],
       [0.29089084, 0.32090822, 0.3882009 ],
       [0.31523523, 0.34776473, 0.33700004],
       [0.4508531 , 0.29054853, 0.2585984 ],
       [0.42230907, 0.3083189 , 0.26937202],
       [0.29438484, 0.30176422, 0.40385097],
       [0.35441118, 0.3160158 , 0.32957304],
       [0.41508636, 0.3201487 , 0.26476496],
       [0.30795678, 0.33973518, 0.35230806],
       [0.39434043, 0.30414775, 0.30151188],
       [0.34434417, 0.3218943 , 0.33376157],
       [0.29964882, 0.33056995, 0.3697812 ],
       [0.35441118, 0.3160158 , 0.32957304],
       [0.

In [125]:
funds = 100
wager = 10
favourites = 0
no_bets = 0
min_diff = 0.03

y_preds = clf.predict_proba(X_test)

for i in range(len(X_test)):
    prediction = np.argmax(y_preds[i])
    print('\nPrediction', prediction)
    print('Actual', y_test[i])
    print('Favourite', np.argmin([testing_data['B365H'][i], testing_data['B365D'][i], 
                                  testing_data['B365A'][i]]))
    print('Prediction proba', y_preds[i])
    print('Home, Draw and Away odds', testing_data['B365H'][i],       
          testing_data['B365D'][i], testing_data['B365A'][i])
    
    if prediction == 0:
        odds_diff = y_preds[i][prediction] - (1/testing_data['B365H'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([testing_data['B365H'][i], testing_data['B365D'][i], 
                                  testing_data['B365A'][i]]):
                favourites +=1
                
            if  prediction == y_test[i]:
                funds += (wager * testing_data['B365H'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
            
    elif prediction == 1:
        odds_diff = y_preds[i][prediction] - (1/testing_data['B365D'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([testing_data['B365H'][i], testing_data['B365D'][i], 
                                  testing_data['B365A'][i]]):
                favourites +=1
            if  prediction == y_test[i]:
                funds +=( wager * testing_data['B365D'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
    else:
        odds_diff = y_preds[i][prediction] - (1/testing_data['B365A'][i])
        if odds_diff >  min_diff:
            if prediction == np.argmin([testing_data['B365H'][i], testing_data['B365D'][i], 
                                  testing_data['B365A'][i]]):
                favourites +=1
            if  prediction == y_test[i]:
                funds += (wager * testing_data['B365A'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
   
    print('Funds', funds)
    
print(f'Betted on favourite {favourites} times out of {len(X_test)} matches.')
print(f'No bet placed {no_bets} times')
         


Prediction 0
Actual 0
Favourite 0
Prediction proba [0.35441118 0.3160158  0.32957304]
Home, Draw and Away odds 1.14 10.0 19.0
Funds 100

Prediction 2
Actual 2
Favourite 2
Prediction proba [0.3104017  0.31818262 0.37141564]
Home, Draw and Away odds 12.0 6.5 1.22
Funds 100

Prediction 1
Actual 1
Favourite 0
Prediction proba [0.31523523 0.34776473 0.33700004]
Home, Draw and Away odds 1.95 3.6 3.6
Funds 126.0

Prediction 2
Actual 1
Favourite 2
Prediction proba [0.30904394 0.34093451 0.35002154]
Home, Draw and Away odds 3.0 3.25 2.37
Funds 126.0

Prediction 0
Actual 2
Favourite 0
Prediction proba [0.34607604 0.3321019  0.32182205]
Home, Draw and Away odds 1.9 3.4 4.0
Funds 126.0

Prediction 0
Actual 0
Favourite 0
Prediction proba [0.35441118 0.3160158  0.32957304]
Home, Draw and Away odds 1.3 5.25 10.0
Funds 126.0

Prediction 1
Actual 1
Favourite 0
Prediction proba [0.31523523 0.34776473 0.33700004]
Home, Draw and Away odds 2.2 3.2 3.4
Funds 148.0

Prediction 2
Actual 2
Favourite 2
Predict

In [120]:
import joblib
#save model
joblib.dump(clf, 'model') 

['model']